<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/qml_300_IsingOnTheCake_template/qml_300_IsingOnTheCake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import pennylane.optimize as optimize

DATA_SIZE = 250

Following the ideas from [this paper](https://www.nature.com/articles/nphys4035), the goal of this challenge is to build a quantum variational classifier to learn the phases of an Ising model with Hamiltonian given by:

$$
H = -\sum_{\langle i, j \rangle} \sigma^z_i \sigma^z_j
$$

The ising model is a classical toy model that describes magnetism via nearest-neighbour inter- acting binary spins. There exists a finite-temperature phase transition for the Ising model where, below the “critical” temperature (the temperature where the phase transition occurs), favoured spin-configurations are all up/down (the “ordered” phase). Above the critical temperature, favoured spin configurations are random (the “disordered” phase).

To do this we simple need to implement a VQE in the function `classify_ising_data` to learn to classify ising configurations as ordered or disordered with more than 90% accuracy

In [ ]:
def square_loss(labels, predictions):
    """Computes the standard square loss between model predictions and true labels.

    Args:
        - labels (list(int)): True labels (1/-1 for the ordered/disordered phases)
        - predictions (list(int)): Model predictions (1/-1 for the ordered/disordered phases)

    Returns:
        - loss (float): the square loss
    """

    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


def accuracy(labels, predictions):
    """Computes the accuracy of the model's predictions against the true labels.

    Args:
        - labels (list(int)): True labels (1/-1 for the ordered/disordered phases)
        - predictions (list(int)): Model predictions (1/-1 for the ordered/disordered phases)

    Returns:
        - acc (float): The accuracy.
    """

    acc = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            acc = acc + 1
    acc = acc / len(labels)

    return acc

In [ ]:
def classify_ising_data(ising_configs, labels):
    """Learn the phases of the classical Ising model.

    Args:
        - ising_configs (np.ndarray): 250 rows of binary (0 and 1) Ising model configurations
        - labels (np.ndarray): 250 rows of labels (1 or -1)

    Returns:
        - predictions (list(int)): Your final model predictions

    Feel free to add any other functions than `cost` and `circuit` within the "# QHACK #" markers 
    that you might need.
    """

    # QHACK #

    num_wires = ising_configs.shape[1] 
    dev = qml.device("default.qubit", wires=num_wires) 

    @qml.qnode(dev)
    def circuit(weights, x):  
        """
        VQE circuit with no bias and inputs:
            weights: weights of the variational circuit
            x: ising configuration data
        """
        qml.BasisState(x, wires=range(num_wires))
        qml.templates.StronglyEntanglingLayers(weights, wires=range(num_wires))
        return qml.expval(qml.PauliZ(0))

    def variational_circuit(weights, bias, x):
        # Variational circuit with bias
        return circuit(weights, x) + bias

    def cost(weights, bias, X, Y):  
        """
        Cost function for the optimization problem with inputs:
            weights: weights of the variational circuit
            bias: bias of the variational circuit
            X: batch of ising configurations
            Y: batch of labels for ising configurations
        """
        # QHACK #
        predictions = [variational_circuit(weights, bias, x) for x in X]
        # QHACK #
        return square_loss(Y, predictions)  # DO NOT MODIFY this line

    # optimize your circuit here
    # Training data
    X = np.array(ising_configs, requires_grad=False)
    Y = np.array(labels, requires_grad=False)
    # Training parameters
    np.random.seed(0)
    num_layers = 3
    learning_rate = 0.1
    weights_init = np.random.uniform(
        low=0, high=2 * np.pi, size=(num_layers, num_wires, 3)
    )
    bias_init = np.array(0.0, requires_grad=True)
    opt = qml.AdamOptimizer(learning_rate)
    batch_size = 10
    epochs = 50
    weights = weights_init
    bias = bias_init

    # Train
    for epoch in range(epochs):
      # Ge training batch
        batch_index = np.random.randint(0, len(X), (batch_size,))
        X_batch = X[batch_index]
        Y_batch = Y[batch_index]
        weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

        # Compute accuracy
        predictions = [int(np.sign(variational_circuit(weights, bias, x))) for x in X]
        acc = accuracy(Y, predictions)
        print(f"Epoch {epoch} -> Accuracy: {acc}")

        # Break once accuracy passes specified threshold
        if acc > 0.9:
            break

    return predictions, acc

Testing 1.in

In [ ]:
inputs = "0,0,1,0,-1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,-1,1,1,0,1,-1,0,0,1,0,-1,1,0,0,1,-1,0,0,0,0,1,1,1,1,1,1,1,0,1,0,-1,0,1,0,1,-1,1,0,0,1,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,-1,0,1,0,1,-1,1,1,1,1,1,0,1,1,0,-1,1,0,1,1,-1,0,0,0,1,-1,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,0,0,1,1,-1,0,0,0,0,1,1,1,1,0,-1,1,1,1,1,1,1,0,1,0,-1,1,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,-1,1,1,1,1,1,0,0,0,1,-1,1,1,1,1,1,1,1,0,1,-1,0,1,0,0,-1,1,1,1,1,1,0,0,1,1,-1,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,-1,0,0,0,0,1,1,1,0,1,-1,0,0,0,0,1,1,0,1,0,-1,1,1,0,1,-1,0,0,0,0,1,1,1,1,1,-1,1,1,1,1,1,1,0,0,1,-1,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,-1,1,0,1,1,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,-1,1,1,1,1,1,0,0,0,0,1,0,0,1,1,-1,0,1,1,1,-1,0,1,1,0,-1,1,0,0,0,-1,1,1,1,1,1,0,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,-1,0,0,0,0,1,1,0,1,0,-1,0,0,0,0,1,1,0,0,1,-1,0,1,1,1,-1,1,0,0,0,-1,0,0,0,0,1,1,1,1,1,-1,1,1,1,0,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,-1,1,0,0,1,-1,0,0,1,1,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,-1,1,1,1,1,1,0,1,0,1,-1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,-1,0,0,1,0,-1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,-1,1,1,1,1,1,0,0,0,0,1,1,0,0,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,-1,0,1,0,0,-1,1,1,1,1,1,0,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,-1,1,1,1,1,1,0,1,0,0,-1,1,1,1,1,1,0,0,1,0,-1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,1,1,0,-1,0,0,0,0,1,0,1,0,0,-1,1,0,1,0,-1,0,1,1,0,-1,1,1,0,0,-1,0,1,1,0,-1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,-1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,-1,0,0,0,0,1,1,0,0,1,-1,1,1,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,1,1,1,1,1,1,0,0,1,-1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,0,0,-1,0,0,0,0,1,1,0,0,1,-1,1,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,1,0,0,1,-1,1,0,0,0,-1,1,0,0,1,-1,1,1,0,1,-1,0,0,0,0,1,0,1,0,1,-1,0,0,1,1,-1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,0,0,-1,0,1,1,1,-1,0,0,0,0,1,1,0,0,0,-1,0,1,1,0,-1,0,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,1,1,0,1,-1,0,0,0,0,1,0,0,1,1,-1,1,1,1,1,-1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,1,1,1,1,1,1,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,1,1,0,1,0,-1,1,0,0,1,-1,0,0,1,1,-1,1,1,0,0,-1,1,0,1,1,-1,0,0,0,0,1,1,1,1,1,1,1,1,0,1,-1,1,1,1,1,1,1,0,0,1,-1,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,1,0,0,1,-1,1,1,1,1,-1,0,0,0,0,-1,1,1,0,0,-1,1,1,1,1,1,1,1,1,1,-1,0,0,0,0,1,0,0,0,1,-1,0,1,1,1,-1,1,1,1,1,1,0,0,0,0,-1,1,0,1,1,-1,1,1,1,0,-1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,-1"
inputs = np.array(
    inputs.split(","), dtype=int, requires_grad=False
).reshape(DATA_SIZE, -1)
ising_configs = inputs[:, :-1]
labels = inputs[:, -1]
predictions, acc = classify_ising_data(ising_configs, labels)
print("Final predictions:")
print(*predictions, sep=",")
print("Final accuracy:")
print(acc)